#### Import required libraries

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pymc as pm

####  Read outputs from the local model

In [4]:
# Read the CSV file containing Y outputs
Y_data = pd.read_csv("Y_out.csv", header=None)

# Extract the first 6 columns (y1 to y6)
Y_values = Y_data.iloc[:, 0:6].values  # Shape: (num_rows, 6)

# Extract the input index from the 7th column (starting from 1 to 10)
input_index = Y_data.iloc[:, 6].values  # Shape: (num_rows,)

# Number of unique input sets
num_samples = len(np.unique(input_index))  # Should be 10

# Number of time steps per input set (given that each set produces 60 rows)
time_steps_per_input = len(Y_values) // num_samples  # Should be 60

# Initialize an array to store time-averaged Y for each input set
Y_samples = np.zeros((num_samples, 6))  # Shape: (10, 6)

# Compute mean response for each input set
for i in range(1, num_samples + 1):  # Input index ranges from 1 to 10
    Y_samples[i - 1, :] = Y_values[input_index == i].mean(axis=0)

# Convert Y_samples into a NumPy array
Y_samples = np.array(Y_samples)  # Shape: (10, 6)

# Print shape to verify
print("Y_samples shape:", Y_samples.shape)  # Expected: (10, 6)


Y_samples shape: (10, 6)


#### Extract correcponsding input values from the input.txt file

In [6]:
# Load the input values (first five columns: Xa1, Xa2, Xe1, Xe2, Xe3)
X_inputs = np.loadtxt("input_new.txt", delimiter=",", usecols=[0, 1, 2, 3, 4])  # Shape: (10, 5)

# Extract the corresponding values
Xa1_vals, Xa2_vals, Xe1_vals, Xe2_vals, Xe3_vals = X_inputs[:, 0], X_inputs[:, 1], X_inputs[:, 2], X_inputs[:, 3], X_inputs[:, 4]

#### Extract dimensions of the input and output data

In [8]:
# Extract number of output dimensions (6 response variables: y1 to y6)
num_outputs = Y_samples.shape[1]
num_samples = X_inputs.shape[0]  # Should be 10

#### Create a PyMC model to estimate the parameters of the model

In [10]:
# Bayesian model
with pm.Model() as bayesian_model:
    # Use observed values directly instead of priors
    Xa1 = pm.Data("Xa1", Xa1_vals)  # Observed aleatory variable
    Xa2 = pm.Data("Xa2", Xa2_vals)
    
    Xe1 = pm.Data("Xe1", Xe1_vals)  # Observed epistemic variable
    Xe2 = pm.Data("Xe2", Xe2_vals)
    Xe3 = pm.Data("Xe3", Xe3_vals)

    # Define the expected mean function for Y (simple linear model assumption)
    mu_y = Xa1 + Xa2 + Xe1 + Xe2 + Xe3  # Can be replaced with a complex function

    # Priors for output noise (uncertainty in model predictions)
    sigma_y = pm.HalfNormal("sigma_y", sigma=1, shape=num_outputs)

    # Likelihood: Y_samples are the observed system responses
    Y_obs = pm.Normal("Y_obs", mu=mu_y, sigma=sigma_y, observed=Y_samples)

#### Perform MCMC sampling

In [13]:
with bayesian_model:
    trace = pm.sample(2000, return_inferencedata=True, cores=2)

ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: ExpandDims{axis=0}(1)
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\Lib\site-packages\pytensor\graph\rewriting\basic.py", line 1920, in process_node
    replacements = node_rewriter.transform(fgraph, node)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\ProgramData\anaconda3\Lib\site-packages\pytensor\graph\rewriting\basic.py", line 1081, in transform
    return self.fn(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\ProgramData\anaconda3\Lib\site-packages\pytensor\tensor\rewriting\basic.py", line 1121, in constant_folding
    thunk = node.op.make_thunk(node, storage_map, compute_map, no_recycling=[])
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\ProgramData\anaconda3\Lib\site

PermissionError: [WinError 5] Access is denied: 'C:\\Users\\hasal\\AppData\\Local\\PyTensor\\compiledir_Windows-11-10.0.26100-SP0-Intel64_Family_6_Model_186_Stepping_3_GenuineIntel-3.12.7-64\\tmp153s7arc'
Apply node that caused the error: ExpandDims{axes=[0, 1]}(-inf)
Toposort index: 0
Inputs types: [TensorType(float32, shape=())]

HINT: Use a linker other than the C linker to print the inputs' shapes and strides.
HINT: Re-running with most PyTensor optimizations disabled could provide a back-trace showing when this node was created. This can be done by setting the PyTensor flag 'optimizer=fast_compile'. If that does not work, PyTensor optimizations can be disabled with 'optimizer=None'.
HINT: Use the PyTensor flag `exception_verbosity=high` for a debug print-out and storage map footprint of this Apply node.